In [ ]:
!pip install prince

In [15]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

# 차원축소
from sklearn.preprocessing import *
from sklearn.decomposition import PCA
import prince


In [35]:
df = pd.read_csv('./data_to_model/세대원_상가_제로_유치원_환경_도시농업_초등학교_사업체_채식_인구수_연령별_카페_유동인구_매출_sum.csv', index_col=0)
df.fillna(0, inplace=True)
df.head(3)

,2023년09월_전체세대,2023년09월_1인세대,2023년09월_2인세대,2023년09월_3인세대,2023년09월_4인세대,2023년09월_5인세대,2023년9월_6인가구이상,채소가게수,제로웨이스트 가게수,유치원수,...,2_40대,2_50대,2_60대,2_70대,2_80대,2_90대,2_기타,카페수,승하차인원_버스_지하철,1점포당_1년_매출액
가락1동,9836,1633,2492,2706,2438,449,118,10.0,0.0,1.0,...,4892,3948,3832,1672,478,94,4,43.0,184724.0,1.175032e+08
가락2동,12375,3568,2746,2825,2641,453,142,3.0,0.0,4.0,...,5304,5224,4686,1834,840,182,8,74.0,190052.0,3.372717e+07
가락본동,11667,4854,2640,2123,1567,385,98,9.0,2.0,2.0,...,3520,4500,4002,1718,768,132,4,99.0,273627.0,3.056880e+07


In [36]:
df.columns

Index(['2023년09월_전체세대', '2023년09월_1인세대', '2023년09월_2인세대', '2023년09월_3인세대',
       '2023년09월_4인세대', '2023년09월_5인세대', '2023년9월_6인가구이상', '채소가게수',
       '제로웨이스트 가게수', '유치원수', '거주인구대비취약자수', '거주인구대비취약자백분위수', '미세먼지지수',
       '미세먼지백분위', '행정구역면적', '공원수', '공원율', '도시농업(프로그램수)', '도시농업(시설수)', '초등학교수',
       '사업체수 (개)', '종사자수 (명)', '채식식당수', '인구수', '1_0대', '1_10대', '1_20대',
       '1_30대', '1_40대', '1_50대', '1_60대', '1_70대', '1_80대', '1_90대', '1_기타',
       '2_0대', '2_10대', '2_20대', '2_30대', '2_40대', '2_50대', '2_60대', '2_70대',
       '2_80대', '2_90대', '2_기타', '카페수', '승하차인원_버스_지하철', '1점포당_1년_매출액'],
      dtype='object')

In [38]:
# 인구 범위 축소하기
df['남자_10대이하'] = df['1_0대'] + df['1_10대']
df['남자_20_30대'] = df['1_20대'] + df['1_30대']
df['남자_40_60대'] = df['1_40대'] + df['1_50대'] + df['1_60대']
df['남자_70대이상'] = df['1_70대'] + df['1_80대'] + df['1_90대'] + df['1_기타']

df['여자_10대이하'] = df['2_0대'] + df['2_10대']
df['여자_20_30대'] = df['2_20대'] + df['2_30대']
df['여자_40_60대'] = df['2_40대'] + df['2_50대'] + df['2_60대']
df['여자_70대이상'] = df['2_70대'] + df['2_80대'] + df['2_90대'] + df['2_기타']

In [39]:
df.columns

Index(['2023년09월_전체세대', '2023년09월_1인세대', '2023년09월_2인세대', '2023년09월_3인세대',
       '2023년09월_4인세대', '2023년09월_5인세대', '2023년9월_6인가구이상', '채소가게수',
       '제로웨이스트 가게수', '유치원수', '거주인구대비취약자수', '거주인구대비취약자백분위수', '미세먼지지수',
       '미세먼지백분위', '행정구역면적', '공원수', '공원율', '도시농업(프로그램수)', '도시농업(시설수)', '초등학교수',
       '사업체수 (개)', '종사자수 (명)', '채식식당수', '인구수', '1_0대', '1_10대', '1_20대',
       '1_30대', '1_40대', '1_50대', '1_60대', '1_70대', '1_80대', '1_90대', '1_기타',
       '2_0대', '2_10대', '2_20대', '2_30대', '2_40대', '2_50대', '2_60대', '2_70대',
       '2_80대', '2_90대', '2_기타', '카페수', '승하차인원_버스_지하철', '1점포당_1년_매출액',
       '남자_10대이하', '남자_20_30대', '남자_40_60대', '남자_70대이상', '여자_10대이하',
       '여자_20_30대', '여자_40_60대', '여자_70대이상'],
      dtype='object')

In [40]:
population_feature = ['2023년09월_1인세대', '남자_10대이하', '남자_20_30대', '남자_40_60대', '남자_70대이상', '여자_10대이하',
       '여자_20_30대', '여자_40_60대', '여자_70대이상']

infra_feature = ['공원수', '공원율', '도시농업(프로그램수)', '도시농업(시설수)']
demand_feature = ['채소가게수', '카페수', '채식식당수', '초등학교수', '유치원수']
environ_featrue = ['미세먼지지수', '미세먼지백분위', '거주인구대비취약자수']
market_feature = ['1점포당_1년_매출액', '승하차인원_버스_지하철', '종사자수 (명)', '사업체수 (개)']

In [41]:
df = df[population_feature + infra_feature + demand_feature + environ_featrue + market_feature]

In [42]:
scaled_data = StandardScaler().fit_transform(np.log1p(df))
data_scale_log=pd.DataFrame(data=scaled_data, columns=df.columns)
data_scale_log

,2023년09월_1인세대,남자_10대이하,남자_20_30대,남자_40_60대,남자_70대이상,여자_10대이하,여자_20_30대,여자_40_60대,여자_70대이상,공원수,...,채식식당수,초등학교수,유치원수,미세먼지지수,미세먼지백분위,거주인구대비취약자수,1점포당_1년_매출액,승하차인원_버스_지하철,종사자수 (명),사업체수 (개)
0,-1.232252,1.244875,0.477965,0.643055,0.257813,1.232333,0.592844,0.683955,-0.122153,-0.522203,...,-1.312350,0.703389,-0.648543,-0.294870,0.471068,0.559375,1.762880,-0.470584,0.473951,0.821430
1,-0.104676,1.196505,0.743152,0.933382,0.512298,1.129477,0.760896,0.960985,0.257900,1.278455,...,-0.264447,0.703389,1.121062,-0.294870,0.471068,0.765070,0.077817,-0.426758,0.400137,0.560240
2,0.339458,0.417238,0.693772,0.641965,0.340402,0.439900,0.612464,0.604165,0.119354,1.278455,...,1.396439,0.703389,0.134520,-0.294870,0.471068,0.277094,-0.054924,0.134980,1.355638,1.234648
3,0.215361,-2.918667,-1.585274,-2.404177,-1.323867,-3.141218,-2.004816,-2.817779,-1.522056,-2.165284,...,-1.312350,-0.255678,-1.987197,-0.375718,0.146494,-0.691571,-1.760553,-0.126329,-0.671353,-0.827389
4,1.876298,-0.733127,1.503474,0.313347,-0.142810,-0.709616,1.289160,-0.243285,-0.422832,-0.091947,...,2.168707,-1.895213,-0.648543,-0.216504,0.684578,-0.510247,1.089504,2.037246,3.358535,3.507110
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421,-1.128324,-0.488349,-0.598500,-0.897183,-1.227573,-0.528896,-0.478198,-0.800295,-0.976641,-0.522203,...,-0.264447,-0.255678,-1.987197,-0.071620,0.896746,-1.263987,0.815518,-0.887482,-1.199103,-1.576227
422,0.147710,-0.165858,0.011571,-0.168064,-0.267126,-0.258045,-0.111101,-0.102322,0.006459,-0.522203,...,0.348536,0.703389,-0.648543,-0.071620,0.896746,-0.341248,0.050870,-0.537470,-0.584872,-0.629070
423,0.277813,-0.500225,0.053614,-0.687745,-0.600937,-0.411399,0.264334,-0.708255,-0.536773,-1.128615,...,0.348536,-1.895213,-0.648543,-0.544693,-0.768429,-0.896944,0.042364,0.728954,-0.822269,-0.717011
424,0.840924,0.366951,0.905033,0.336318,0.416288,0.435516,0.683939,0.306523,0.414241,0.241786,...,0.783455,0.703389,0.690111,-0.544693,-0.768429,0.259657,-0.631576,-0.151431,-0.402938,-0.348691


#### 인구특성



In [43]:
population_feature = ['2023년09월_1인세대', '남자_10대이하', '남자_20_30대', '남자_40_60대', '남자_70대이상', '여자_10대이하',
       '여자_20_30대', '여자_40_60대', '여자_70대이상']

pca = PCA(n_components=0.9)
population_scale_data = data_scale_log[population_feature]
pca.fit(population_scale_data)
pca_population = pd.DataFrame((pca.transform(population_scale_data)))

num_of_principal = pca_population.shape[1] 
print('주성분의 개수:', num_of_principal)

주성분의 개수: 2


In [44]:
print('주성분의 설명력')
for i in range(num_of_principal) :
    print(f"제 {i+1}주성분 :", pca.explained_variance_ratio_[i])
    
pd.DataFrame(data=pca.components_,columns=population_scale_data.columns).T

주성분의 설명력
제 1주성분 : 0.8243068415827797
제 2주성분 : 0.11777893165341181


,0,1
2023년09월_1인세대,-0.248212,0.691402
남자_10대이하,-0.323765,-0.422133
남자_20_30대,-0.344253,0.268372
남자_40_60대,-0.358644,-0.102116
남자_70대이상,-0.345005,0.038929
여자_10대이하,-0.323410,-0.419514
여자_20_30대,-0.342574,0.232359
여자_40_60대,-0.357236,-0.168928
여자_70대이상,-0.343270,0.035784


### 환경 관련 인프라 특성

In [45]:
infra_feature = ['공원수', '공원율', '도시농업(프로그램수)', '도시농업(시설수)']

pca = PCA(n_components=0.9)
infra_scale_data = data_scale_log[infra_feature]
pca.fit(infra_scale_data)
pca_infra = pd.DataFrame((pca.transform(infra_scale_data)))

num_of_principal = pca_infra.shape[1] 
print('주성분의 개수:', num_of_principal)

주성분의 개수: 3


In [46]:
print('주성분의 설명력')
for i in range(num_of_principal) :
    print(f"제 {i+1}주성분 :", pca.explained_variance_ratio_[i])
    
pd.DataFrame(data=pca.components_,columns=infra_scale_data.columns).T

주성분의 설명력
제 1주성분 : 0.5128415636874609
제 2주성분 : 0.33649128739829876
제 3주성분 : 0.1421796835550979


,0,1,2
공원수,0.270214,0.644683,-0.715086
공원율,0.213704,0.684016,0.697345
도시농업(프로그램수),0.662950,-0.245483,0.024694
도시농업(시설수),0.664690,-0.237158,0.041869


### 예상 수요 특성

In [47]:
demand_feature = ['채소가게수', '카페수', '채식식당수', '초등학교수', '유치원수']

pca = PCA(n_components=0.9)
demand_scale_data = data_scale_log[demand_feature]
pca.fit(demand_scale_data)
pca_demand = pd.DataFrame((pca.transform(demand_scale_data)))

num_of_principal = pca_demand.shape[1] 
print('주성분의 개수:', num_of_principal)

주성분의 개수: 4


In [48]:
print('주성분의 설명력')
for i in range(num_of_principal) :
    print(f"제 {i+1}주성분 :", pca.explained_variance_ratio_[i])
    
pd.DataFrame(data=pca.components_,columns=demand_scale_data.columns).T

주성분의 설명력
제 1주성분 : 0.36276318573145905
제 2주성분 : 0.30545785029050854
제 3주성분 : 0.1892536332273014
제 4주성분 : 0.09390414272839434


,0,1,2,3
채소가게수,-0.306171,-0.009671,0.928065,0.131210
카페수,-0.689894,0.043408,-0.088282,-0.078886
채식식당수,-0.655775,-0.033026,-0.337349,-0.001372
초등학교수,-0.007096,0.705509,-0.091342,0.702558
유치원수,0.014627,0.706533,0.093567,-0.694961


### 환경 오염 특성

In [49]:
environ_feature = ['미세먼지지수', '미세먼지백분위', '거주인구대비취약자수']

pca = PCA(n_components=0.9)
environ_scale_data = data_scale_log[environ_feature]
pca.fit(environ_scale_data)
pca_environ = pd.DataFrame((pca.transform(environ_scale_data)))

num_of_principal = pca_environ.shape[1] 
print('주성분의 개수:', num_of_principal)

주성분의 개수: 2


In [50]:
print('주성분의 설명력')
for i in range(num_of_principal) :
    print(f"제 {i+1}주성분 :", pca.explained_variance_ratio_[i])
    
pd.DataFrame(data=pca.components_,columns=environ_scale_data.columns).T

주성분의 설명력
제 1주성분 : 0.6214892505570294
제 2주성분 : 0.29684237358975796


,0,1
미세먼지지수,0.676542,-0.145958
미세먼지백분위,0.644634,-0.356461
거주인구대비취약자수,-0.356003,-0.922839


### 상권 특성

In [51]:
market_feature = ['1점포당_1년_매출액', '승하차인원_버스_지하철', '종사자수 (명)', '사업체수 (개)']

pca = PCA(n_components=0.9)
market_scale_data = data_scale_log[market_feature]
pca.fit(market_scale_data)
pca_market = pd.DataFrame((pca.transform(market_scale_data)))

num_of_principal = pca_market.shape[1] 
print('주성분의 개수:', num_of_principal)

주성분의 개수: 3


In [52]:
print('주성분의 설명력')
for i in range(num_of_principal) :
    print(f"제 {i+1}주성분 :", pca.explained_variance_ratio_[i])
    
pd.DataFrame(data=pca.components_,columns=market_scale_data.columns).T

주성분의 설명력
제 1주성분 : 0.6305867737478807
제 2주성분 : 0.23598886514335926
제 3주성분 : 0.10662930670362157


,0,1,2
1점포당_1년_매출액,-0.194593,0.978359,0.036117
승하차인원_버스_지하철,-0.521853,-0.072574,-0.849939
종사자수 (명),-0.590866,-0.087148,0.368123
사업체수 (개),-0.583678,-0.173068,0.375214


## 차원축소 결과

In [53]:
pca_mca=pd.concat([pca_population, pca_infra, pca_demand, pca_environ, pca_market],axis=1)

In [54]:
# pca_mca.columns=['']

pca_mca.head()

,0,1,0,1,2,0,1,2,3,0,1,0,1,2
0,-1.385345,-1.803981,-0.665221,-0.359604,0.154651,0.236120,0.070813,1.875716,1.158075,-0.094965,-0.641092,-0.856961,1.575414,0.946321
1,-2.186509,-0.903525,0.346363,2.481719,0.580242,-0.475947,1.330384,0.320708,-0.312788,-0.168193,-0.830915,-0.355864,-0.024727,0.723038
2,-1.414738,0.052435,-0.017745,1.316296,-0.607890,-2.229353,0.582526,0.817341,0.508024,0.005528,-0.380592,-1.581389,-0.395351,0.845590
3,5.988013,2.422976,-1.254477,-1.883855,0.855550,1.654339,-1.617270,1.205942,1.438117,0.086447,0.640829,1.288125,-1.511582,-0.513801
4,-0.789125,2.595896,-0.671977,-0.475974,-0.554440,-3.424138,-1.776846,0.331809,-0.905590,0.476478,0.258452,-5.306620,0.018415,0.860085
